## Controlling webcam

In [ ]:
import numpy as np
import cv2 as cv
from PIL import Image
import ipywidgets
from IPython.display import clear_output

In [ ]:
from io import BytesIO
import IPython.display
import PIL.Image

SOURCE = "usb" # usb, csi

def get_source(id, source=SOURCE):
    if SOURCE == "usb":
        return id
    elif SOURCE == "csi":
        return (
            "nvarguscamerasrc sensor-id=%d sensor-mode=%d ! "
            "video/x-raw(memory:NVMM), "
            "width=(int)%d, height=(int)%d, "
            "format=(string)NV12, framerate=(fraction)%d/1 ! "
            "nvvidconv flip-method=%d ! "
            "video/x-raw, width=(int)%d, height=(int)%d, format=(string)BGRx ! "
            "videoconvert ! "
            "video/x-raw, format=(string)BGR ! appsink"
            % (
                id, # camera id
                3, # mode
                640, # width
                480, # height
                10, # frame rate
                0, # flip method
                640, # display width
                480, # display height
            )
        )
    else:
        raise Exception("Unknown camera source.")

def camera_capture(id):
    camera = cv.VideoCapture(get_source(id))
    camera.set(3, 1280)
    camera.set(4, 1024)
    ret, image = camera.read()

    if not ret:
        return None

    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    camera.release()
    return image

def camera_stream(id):
    camera = cv.VideoCapture(get_source(id))
    camera.set(3, 1280)
    camera.set(4, 1024)
    try: 
        while True:
            ret, frame = camera.read()
            frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

            frame = cv.rotate(frame, cv.ROTATE_90_COUNTERCLOCKWISE)

            stream = BytesIO()
            PIL.Image.fromarray(frame).save(stream, format="jpeg")
            IPython.display.display(IPython.display.Image(data=stream.getvalue()))

            IPython.display.clear_output(wait=True)
    except KeyboardInterrupt:
        camera.release()
        print("Stream Stopped")
        
def camera_stream2():
    camera0 = cv.VideoCapture(get_source(0))
    camera0.set(3, 640)
    camera0.set(4, 480)
    
    camera1 = cv.VideoCapture(get_source(1))
    camera1.set(3, 640)
    camera1.set(4, 480)
    try: 
        while True:
            ret1, frame1 = camera1.read()
            ret0, frame0 = camera0.read()
            frame0 = cv.cvtColor(frame0, cv.COLOR_BGR2GRAY)
            frame1 = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
            
            frame = np.hstack((frame0, frame1))
            
            frame = cv.flip(frame, 1)

            stream = BytesIO()
            PIL.Image.fromarray(frame).save(stream, format="jpeg")
    
            IPython.display.display(IPython.display.Image(data=stream.getvalue()))
            IPython.display.clear_output(wait=True)
    except KeyboardInterrupt:
        camera0.release()
        camera1.release()
        print("Stream Stopped")

## Calibration

### Collecting images

In [ ]:
import os

chess = cv.rotate(camera_capture(0), cv.ROTATE_90_COUNTERCLOCKWISE)
count = len(os.listdir("calibration"))
Image.fromarray(chess).save(os.path.join("calibration", str(count) + ".jpg"))
display(Image.fromarray(chess))

### Loading images

In [ ]:
images = []
for f in os.listdir("calibration"):
    image = cv.imread(os.path.join("calibration", f))
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    images.append(image)

print("Images loaded:", len(images))

### Finding chessboard corners

In [ ]:
obj_points = []
img_points = []

for idx, image in enumerate(images):
    print("Processing image", idx, end=": ")
    ret, corners = cv.findChessboardCorners(image, (7, 7), None)

    image = cv.drawChessboardCorners(image, (7, 7), corners, ret)
    Image.fromarray(image).save(os.path.join("calibration-corners", str(idx) + ".jpg"))

    if not ret:
        print("ERROR")
        continue
    
    obj = np.zeros((7 * 7, 3), np.float32)
    obj[:, :2] = np.mgrid[0:7, 0:7].T.reshape(-1, 2)
    obj_points.append(obj)

    img_points.append(corners)

    print("OK")

### Calculating distortions

In [ ]:
ret, matrix, distortion_coeff, rotation_vec, translation_vec = cv.calibrateCamera(obj_points, img_points, (1024, 1280), None, None)
new_camera_matrix, roi = cv.getOptimalNewCameraMatrix(matrix, distortion_coeff, (1024, 1280), 1, (1024, 1280))

In [ ]:
for idx, image in enumerate(images):
    image_undist = cv.undistort(image, matrix, distortion_coeff, None, new_camera_matrix)
    Image.fromarray(image_undist).save(os.path.join("calibration-undistorted", str(idx) + ".jpg"))

    x, y, w, h = roi
    image_undist_roi = image_undist[y:y + h, x:x + w]
    Image.fromarray(image_undist_roi).save(os.path.join("calibration-undistorted-roi", str(idx) + ".jpg"))

### Calculating re-projection error

In [ ]:
mean_error = 0
for i in range(len(obj_points)):
    img_points2, _ = cv.projectPoints(obj_points[i], rotation_vec[i], translation_vec[i], matrix, distortion_coeff)
    mean_error += cv.norm(img_points[i], img_points2, cv.NORM_L2) / len(img_points2)

print("Total error:", mean_error / len(obj_points))

### Saving coefficients

In [ ]:
np.savez("logitech-500.npz", 
    matrix=matrix,
    distortion_coeff=distortion_coeff,
    rotation_vec=rotation_vec,
    translation_vec=translation_vec,
    new_camera_matrix=new_camera_matrix,
    roi=roi)

coefficients = dict(np.load("logitech-500.npz"))

## Pose Estimation

In [ ]:
coefficients = dict(np.load("logitech-500.npz"))
matrix, distortion_coeff = coefficients["matrix"], coefficients["distortion_coeff"]

In [ ]:
def draw(img, corners, imgpts):
    corner = tuple(corners[0].ravel())
    img = cv.line(img, corner, tuple(imgpts[0].ravel()), (255,0,0), 5)
    img = cv.line(img, corner, tuple(imgpts[1].ravel()), (0,255,0), 5)
    img = cv.line(img, corner, tuple(imgpts[2].ravel()), (0,0,255), 5)
    return img

In [ ]:
camera = cv.VideoCapture(get_source(0))
camera.set(3, 640)
camera.set(4, 480)
try: 
    while True:
        obj_points = []
        img_points = []

        ret, frame = camera.read()
        frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        frame = cv.rotate(frame, cv.ROTATE_90_COUNTERCLOCKWISE)

        ret, corners = cv.findChessboardCorners(frame, (7,7), None)
        if ret:
            obj = np.zeros((7 * 7, 3), np.float32)
            obj[:, :2] = np.mgrid[0:7, 0:7].T.reshape(-1, 2)
            obj_points.append(obj)
            ret, rvecs, tvecs = cv.solvePnP(obj, corners, matrix, distortion_coeff)

            axis = np.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)
            img_points, jac = cv.projectPoints(axis, rvecs, tvecs, matrix, distortion_coeff)

            frame = cv.cvtColor(frame, cv.COLOR_GRAY2RGB)
            frame = draw(frame, corners, img_points)

        stream = BytesIO()
        PIL.Image.fromarray(frame).save(stream, format="jpeg")
        IPython.display.display(IPython.display.Image(data=stream.getvalue()))

        IPython.display.clear_output(wait=True)
except KeyboardInterrupt:
    camera.release()
    print("Stream Stopped")

## Depth map

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
ID_LEFT = 0
ID_RIGHT = 1

# left, right = camera_capture(ID_LEFT), camera_capture(ID_RIGHT)
left, right = cv.imread("left.jpg"), cv.imread("right.jpg")
left, right = cv.cvtColor(left, cv.COLOR_BGR2GRAY), cv.cvtColor(right, cv.COLOR_BGR2GRAY)

stereo = cv.StereoBM_create(numDisparities=16, blockSize=5)
disparity = stereo.compute(left, right)

plt.subplots(figsize=(20, 20))
plt.imshow(disparity)

In [ ]:
left = cv.rotate(camera_capture(0), cv.ROTATE_90_COUNTERCLOCKWISE)
Image.fromarray(left).save("left.jpg")
display(Image.fromarray(left))

In [ ]:
right = cv.rotate(camera_capture(0), cv.ROTATE_90_COUNTERCLOCKWISE)
Image.fromarray(right).save("right.jpg")
display(Image.fromarray(right))

In [ ]:
coefficients = dict(np.load("logitech-500.npz"))
matrix, distortion_coeff, new_camera_matrix = coefficients["matrix"], coefficients["distortion_coeff"], coefficients["new_camera_matrix"]

In [ ]:
left_undist = cv.undistort(left, matrix, distortion_coeff, None, new_camera_matrix)

x, y, w, h = roi
left_undist = left_undist[y:y + h, x:x + w]

display(Image.fromarray(left_undist))

In [ ]:
right_undist = cv.undistort(right, matrix, distortion_coeff, None, new_camera_matrix)

x, y, w, h = roi
right_undist = right_undist[y:y + h, x:x + w]

display(Image.fromarray(right_undist))

In [ ]:
stereo = cv.StereoBM_create(numDisparities=16, blockSize=15)
disparity = stereo.compute(left, right)

plt.subplots(figsize=(20, 20))
plt.imshow(disparity)